In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.express as px

# 1.NYT debt ceiling data

In [46]:
dir = os.getcwd()
df_debt = pd.read_csv(dir + '/debt_ceiling_nyt.csv')
df_debt = df_debt.sort_values(by = "pub_date")

df_debt_may = pd.read_csv(dir + '/debt_ceiling_nyt_may.csv')
df_debt_may = df_debt_may.sort_values(by = "pub_date")

df_debt = pd.concat([df_debt, df_debt_may])

In [47]:
# convert headline to dic and then select only main
df_debt['headline'] = df_debt['headline'].apply(eval)
df_debt['headline'] = df_debt['headline'].apply(lambda x: x['main'])

In [48]:
df_debt_2023 = df_debt[df_debt['pub_date'].str.contains('2023')]

## 1.0 update data in May

In [ ]:
# update data in May

import requests

# API key
api_key = 'PYV795sFJXDJShcDk2z8GdAA8lTExWGb'
url_may = "https://api.nytimes.com/svc/archive/v1/2023/5.json?api-key=" + api_key
#get data
data_may = requests.get(url_may).json()
df_may = pd.DataFrame(data_may['response']['docs'])

df_debt_may = df_may[df_may['keywords'].apply(lambda x: 'National Debt (US)' in str(x))]
df_debt_may['pub_date'] = pd.to_datetime(df_debt_may['pub_date']).dt.date

# save to csv
df_debt_may.to_csv('/Users/yjy/Documents/GitHub/FinAI/case_study/debt_ceiling/debt_ceiling_nyt_may.csv', index=False)

## 1.1 trend

In [49]:
# plot the number of articles for each day, sort by date
px.line(df_debt['pub_date'].value_counts().sort_index())

In [50]:
#plot the curve for 2023 using plotly
px.line(df_debt_2023['pub_date'].value_counts().sort_index())

## 1.2 section

In [51]:
# values for document_type,type_of_material,news_desk,section_name,subsection_name
print(df_debt['document_type'].value_counts())
# insert a line to separate the output
print('---------------------------')
print(df_debt['type_of_material'].value_counts())
print('---------------------------')
print(df_debt['news_desk'].value_counts())
print('---------------------------')
print(df_debt['section_name'].value_counts())
print('---------------------------')
print(df_debt['subsection_name'].value_counts())

article       1185
multimedia      61
Name: document_type, dtype: int64
---------------------------
News                   816
Op-Ed                  202
Editorial               56
Letter                  41
Interactive Feature     36
News Analysis           33
Video                   24
Question                 4
briefing                 4
Brief                    2
Series                   1
Slideshow                1
Review                   1
Name: type_of_material, dtype: int64
---------------------------
National           259
Washington         175
Editorial          136
Business           132
OpEd               128
Letters             37
U.S.                29
SundayBusiness      26
Politics            12
NAT                  7
Foreign              7
Opinion              5
Upshot               5
NYTNow               4
Podcasts             4
Business Day         4
Metro                3
Magazine             3
Insider              2
The Upshot           2
NewsDesk             2
C

## 1.3 keyword

In [52]:
#select pub_date, headline, abstrct and lead_paragraph
df_debt_text = df_debt[['pub_date', 'headline', 'abstract', 'lead_paragraph']]
df_debt_text

,pub_date,headline,abstract,lead_paragraph
217,2010-01-22,Debt Burden Now Rests More on U.S. Shoulders,China lent an estimated 4.6 percent of the mon...,THE United States government borrowed more mon...
218,2010-01-25,Independent Group to Look at Ways to Reduce Debt,The group could provide pressure and political...,WASHINGTON  Just as President Obama and Congr...
220,2010-01-26,President Plans Own Panel on the Debt,A plan for a bipartisan commission to press fo...,WASHINGTON  Advocates of more aggressive step...
219,2010-01-26,Obama to Seek Spending Freeze to Trim Deficits,President Obama will call for a three-year fre...,WASHINGTON  President Obama will call for a t...
222,2010-01-27,Turning the Focus to Fiscal Restraint,The announcement of a limited spending freeze ...,The announcement of a limited spending freeze ...
...,...,...,...,...
65,2023-05-19,Rational Approaches to Immigration,Readers offer suggestions on U.S. policy. Also...,To the Editor:
66,2023-05-19,Whiplash Day of Debt Limit Talks Ends Without ...,Republicans returned to the negotiating table ...,Bipartisan talks between top White House and R...
68,2023-05-19,Finger-Pointing Won’t Save Anyone if Default L...,While each party tries to blame the other for ...,Is it the Biden default? Or the Republican Def...
69,2023-05-20,Biden Voices Optimism on Debt Talks Despite Sh...,He said the recent harsh back-and-forth was po...,President Biden on Saturday brushed off noisy ...


In [68]:
# count "debt ceiling" or debt limit in any column
df_debt_text.apply(lambda x: x.str.contains('debt ceiling|debt limit|debt crisis', case=False).any(), axis=1).value_counts()

False    729
True     517
dtype: int64

In [54]:
# count "debt" in any column
df_debt_text.apply(lambda x: x.str.contains('debt', case=False).any(), axis=1).value_counts()

True     808
False    438
dtype: int64

In [55]:
df_debt_text.apply(lambda x: x.str.contains('national debt|federal debt', case=False).any(), axis=1).value_counts()

False    1084
True      162
dtype: int64

# 2. Melwater

In [25]:
dir = os.getcwd()
df_debt_mw = pd.read_csv(dir + '/debt_ceiling_meltwater.csv')

#remove time from "Date" and convert to YYYY-MM-DD format
df_debt_mw['Date'] = df_debt_mw['Date'].apply(lambda x: x.split(' ')[0])
df_debt_mw['Date'] = pd.to_datetime(df_debt_mw['Date'])
df_debt_mw['Date'] = df_debt_mw['Date'].dt.strftime('%Y-%m-%d')

df_debt_mw

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,...,Twitter Bio,Twitter Followers,Twitter Following,Alternate Date Format,Time,State,City,Views,Parent URL,Document Tags
0,2023-05-21,U.S. Default Prospect Hurts Economy in the Mea...,https://www.nytimes.com/2023/05/20/business/ec...,As negotiations over the debt limit continue i...,... to new needs and crises.The economic turbu...,The New York Times,Lydia DePillis,United States,New York,English,...,NaN,NaN,NaN,21-May-23,7:00 AM,New York,New York City,NaN,NaN,NaN
1,2023-05-21,Biden Says He’s Optimistic on Debt Talks,https://www.nytimes.com/2023/05/20/us/politics...,"HIROSHIMA, Japan — President Biden on Saturday...",... spending constraints as part of an agreeme...,The New York Times,Peter Baker,United States,New York,English,...,NaN,NaN,NaN,21-May-23,1:12 AM,New York,New York City,NaN,NaN,NaN
2,2023-05-20,Whiplash Day of Debt Limit Talks Ends Without ...,https://www.nytimes.com/2023/05/19/us/politics...,WASHINGTON — Bipartisan talks between top Whit...,... not make a deal on the debt ceiling unless...,The New York Times,Catie Edmondson,United States,New York,English,...,NaN,NaN,NaN,20-May-23,10:08 AM,New York,New York City,NaN,NaN,NaN
3,2023-05-20,"If Debt Ceiling Standoff Can’t Be Resolved, Bo...",https://www.nytimes.com/2023/05/19/us/politics...,WASHINGTON — Is it the Biden default? Or the R...,... things go south. Democrats lambaste Republ...,The New York Times,Carl Hulse,United States,New York,English,...,NaN,NaN,NaN,20-May-23,7:44 AM,New York,New York City,NaN,NaN,NaN
4,2023-05-20,"If Nation Defaults, Both Parties Will Share th...",https://www.nytimes.com/2023/05/19/us/politics...,WASHINGTON — Is it the Biden default? Or the R...,... things go south. Democrats lambaste Republ...,The New York Times,Carl Hulse,United States,New York,English,...,NaN,NaN,NaN,20-May-23,5:49 AM,New York,New York City,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,2023-01-06,Republicans’ Internecine Conflict Is Mirrored ...,https://www.nytimes.com/2023/01/05/business/me...,"William F. Buckley Jr., a founder of National ...",Would you not rather have Trump leading the ne...,The New York Times,Jeremy W. Peters,United States,New York,English,...,NaN,NaN,NaN,06-Jan-23,6:59 AM,New York,New York City,NaN,NaN,NaN
255,2023-01-05,Biden Promotes Compromise at a Moment of Repub...,https://www.nytimes.com/2023/01/04/us/politics...,"COVINGTON, Ky. — President Biden on Wednesday ...",... him to help cut a deal with Republicans on...,The New York Times,Michael D. Shear,United States,New York,English,...,NaN,NaN,NaN,05-Jan-23,10:11 AM,New York,New York City,NaN,NaN,NaN
256,2023-01-05,What Will Republicans Do With Their House Majo...,https://www.nytimes.com/2023/01/04/opinion/rep...,This article is part of the Debatable newslett...,... is expected to hit the debt ceiling once a...,The New York Times,Spencer Bokat-Lindell,United States,New York,English,...,NaN,NaN,NaN,05-Jan-23,7:00 AM,New York,New York City,NaN,NaN,NaN
257,2023-01-04,What a Failed Speaker Vote Means for Kevin McC...,https://www.nytimes.com/2023/01/02/opinion/kev...,"News update: On Tuesday, Representative Kevin ...",... the government and preventing a credit def...,The New York Times,Brendan Buck,United States,New York,English,...,NaN,NaN,NaN,04-Jan-23,11:49 PM,New York,New York City,NaN,NaN,NaN


In [56]:
px.line(df_debt_mw['Date'].value_counts().sort_index())

In [57]:
# NYT API
px.line(df_debt_2023['pub_date'].value_counts().sort_index())

# 3. Text Analysis (May 2023)
- filter news by keyword mapping
- analysis all headlines/abstract
- define tracking terms/scenarios
- calculate similarity of news and scenarios

In [69]:
#select pub_date, headline, abstrct and lead_paragraph
#select text in may for df_debt_2023
df_debt_may = df_debt_2023[df_debt_2023['pub_date'].str.contains('2023-05|2023-04')]
df_debt_may_text = df_debt_may[['pub_date', 'headline', 'abstract', 'lead_paragraph']]
df_debt_may_text

,pub_date,headline,abstract,lead_paragraph
22,2023-04-03,Republicans Can’t Agree on a Path Out of Their Own Debt Crisis,House Republicans have become so factionalized that their leaders cannot speak in one voice.,"It’s been obvious for months that Republicans in the House of Representatives intend to lead the country toward a calamitous debt ceiling crisis in order to get their way on spending. Only in the last few weeks, however, has it become clear just how truly chaotic that process will be."
23,2023-04-06,"Staring Down a Debt Crisis, McCarthy Toils to Navigate G.O.P. Divisions",The House speaker has little trust in his Budget Committee chairman or his No. 2 as he works to unite the fractious Republican majority around a spending plan.,"WASHINGTON — It was midway through Representative Kevin McCarthy’s drawn-out battle for the House speakership when Representative Jodey C. Arrington of Texas, one of his public supporters, began quietly approaching colleagues to see whether they would be open to backing his No. 2, Representative Steve Scalise of Louisiana, instead."
24,2023-04-07,Can Congress Make an End-Run Around a Debt Limit Impasse? It’s Tricky.,Some Democrats are urging their colleagues to lay the groundwork for using an arcane procedural process to bypass Republicans and stave off economic peril.,"WASHINGTON — Call it an escape valve, an off-ramp or a break-glass-in-case-of-emergency option."
25,2023-04-17,McCarthy Proposes One-Year Debt Ceiling Increase Tied to Spending Cuts,"Clashing with President Biden, who has called for raising the debt limit with no conditions, the Republican speaker went to Wall Street to pitch what he called a “responsible, sensible” solution.","WASHINGTON — Speaker Kevin McCarthy on Monday proposed a one-year debt ceiling increase paired with a set of spending cuts and policy changes, backing down substantially from earlier demands but making clear that Republicans would not raise the borrowing limit to avert a catastrophic debt default without conditions."
28,2023-04-18,The G.O.P.’s Fiscal Hawks Fly Far Away From Deficit Fights,"After a decade of rising deficits and soaring debt, the top White House contenders, Donald Trump and Ron DeSantis, show little interest in battling over the nation’s finances.","The first skirmish of the Republican presidential primary of 2024 broke through this weekend. It was not over a traditional theme of conservative politics, such as national defense, or more contemporary issues like immigration or “woke” social policy."
...,...,...,...,...
65,2023-05-19,Rational Approaches to Immigration,Readers offer suggestions on U.S. policy. Also: The debt negotiations; Senator Dianne Feinstein; urban exodus; aid workers in Sudan.,To the Editor:
66,2023-05-19,Whiplash Day of Debt Limit Talks Ends Without a Breakthrough,"Republicans returned to the negotiating table on Friday evening after briefly walking out, though the evening talks lasted only an hour.","Bipartisan talks between top White House and Republican congressional officials over a deal to raise the debt limit ended without a breakthrough on Friday, capping a day of whiplash as negotiators seeking to avoid the first default in the nation’s history repeatedly started and ended discussions amid growing G.O.P. frustration."
68,2023-05-19,Finger-Pointing Won’t Save Anyone if Default Leads to Economic Collapse,"While each party tries to blame the other for the crisis, some acknowledge that they would both share responsibility for a default.",Is it the Biden default? Or the Republican Default on America?
69,2023-05-20,Biden Voices Optimism on Debt Talks Despite Sharp Statements by Both Sides,He said the recent harsh back-and-forth was posturing typical of any negotiation and predicted the two sides would still “get something decent done.”,"President Biden on Saturday brushed off noisy statements issued by both sides in the debt and spending talks gripping Washington, dismissing them as little more than the posturing ty

In [70]:
# show full text of headline
pd.set_option('display.max_colwidth', None)
df_debt_may_text

# to csv
df_debt_may_text.to_csv('/Users/yjy/Documents/GitHub/FinAI/case_study/debt_ceiling/headline_may.csv', index=False)

In [71]:
# concat all headline in df_debt_may_text
headlines = df_debt_may_text['headline'].str.cat(sep='\\')


In [72]:
import openai
from openai.embeddings_utils import cosine_similarity

In [73]:
# topic extraction
def text_anlysis(prompt,text):

    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": "You are a helpful assistant that summarize and analyze text."},
            {"role": "user", "content": prompt + text},
        ]
    )

    return response

In [76]:
prompt = """Based on the text of headlines of news about US debt ceiling crisis in 2023, answer:
1.summarise all possible scenarios or solutions to the crisis.
2.what is probability for each scenarios?
3.what kinds of attitudes do people have towards each scenarios?

Text:
"""

In [77]:
response = text_anlysis(prompt,headlines)

In [87]:
print(response["choices"][0]["message"]["content"])

1. Possible scenarios or solutions to the US debt ceiling crisis include proposals to increase the debt ceiling alongside spending cuts, the use of various legal workarounds (such as invoking the 14th Amendment of the Constitution), and negotiations between President Biden and Republican leaders. House Democrats have also proposed using a petition to force a debt limit vote.
2. The probability for each scenario is difficult to determine as they are subject to factors such as political negotiations and the timeline for reaching an agreement before the US runs out of money. However, proposals that involve compromise from both parties may have a higher probability of success.
3. Attitudes towards each scenario are varied, with some supporting spending cuts and others advocating for more leniency in increasing the debt ceiling. The use of legal workarounds like invoking the 14th amendment is also a controversial topic, with some questioning the constitutionality of the move. Additionally, 

# 4. Another case: Inflation

# try: select all news with "inflation" in title/abstract/leading paragragh and keyword?